### For HP sustainability track - chemical search engine

Done by group PROcrastination - code by Luo Wenyu and Wei Yao, requirement analysis done by Chen Xiaofei

In [1]:
#import python library 
import numpy as np
import pandas as pd
import dask.dataframe as dd

In [2]:
#extract all tables from GSE
import pdfplumber
from PyPDF2 import PdfFileReader
pdf = pdfplumber.open("HPGSE.pdf")
table_list= list()
HPpage = PdfFileReader(open('HPGSE.pdf','rb')).getNumPages()
for i in range(HPpage):
    p0 = pdf.pages[i]
    try:
        table2df = lambda table: pd.DataFrame(table[1:], columns=table[0])
        tables = table2df(p0.extract_table())
        table_list.append(tables)
    except TypeError:
        continue

In [3]:
# combine table with same heading
def combine(df, leng):
    i = 0
    while i < leng:
        try:
            if df[i].columns[0] == df[i+1].columns[0]:
                df[i] = pd.concat([df[i], df[i+1]])
                del df[i+1]
                new = df
                combine(new,len(new))
            else:
                i+=1
        except IndexError:
            break
combine(table_list, len(table_list))

#filter irrelevant tables by keywords
String = ['Document','Identifier','Revision','Change','nDate','change','document','identifier','Revised','•']
relevant = []
for table in table_list:
    try:
        if all(ele not in table.columns for ele in String):
            relevant.append(table)
      #  any(ele in table.columns[1] for ele in String) == False:
       #     relevant.append(table)
    except TypeError:
        continue

#test = table_list[5]
#.drop_duplicates()
test.reset_index(inplace = False)
#table_list[5]['Table 1. Pan HP Mandatory Restrictions for All Products']
#for column in table_list[5]:
    #table_list[5][column] = table_list[5][column].str.replace('\n', ' ')
    #print(table_list[5][column])
#test

In [7]:
#import google translate library 
from google_trans_new import google_translator
translator = google_translator()  

In [8]:
chemCH = pd.read_excel('testinput.xlsx')
chemCH['有害物质名称'] = chemCH['有害物质名称'].fillna(method='ffill')
chemCH['Unnamed: 1'] = chemCH['Unnamed: 1'].fillna(chemCH['有害物质名称'])

In [9]:
#append translated columns to table in chinese
category=[]
name_sub=[]
limitation = []
for i in range(len(chemCH)):
    cate=translator.translate(chemCH['有害物质名称'][i],lang_tgt='en')
    name=translator.translate(chemCH['Unnamed: 1'][i],lang_tgt='en')
    limit=translator.translate(chemCH['限  值'][i],lang_tgt='en')
    category.append(cate)
    name_sub.append(name)
    limitation.append(limit)
#rename columns
chemCH['Category'] = category
chemCH['Name'] = name_sub
chemCH['Limitation'] = limitation
chemCH['tMethod'] = chemCH['测试方法']

In [11]:
chemJP = pd.read_excel('jap1-2.xlsx')
name_sub_jp=[]
limitation_jp = []
for i in range(len(chemJP)):
    nsjp=translator.translate(chemJP['対象物賀名'][i],lang_tgt='en')
    ljp=translator.translate(chemJP['閾値（法規制値）'][i],lang_tgt='en')
    name_sub_jp.append(nsjp)
    limitation_jp.append(ljp)
chemJP['Name'] = name_sub_jp
chemJP['Limitation'] = limitation_jp

In [13]:
#cosine similarity
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
def cosine_sim_vectors(vec1,vec2):
    vec1 = vec1.reshape(1,-1)
    vec2 = vec2.reshape(1,-1)
    return cosine_similarity(vec1, vec2)[0][0]

i = 0
result = []
while i < len(chemCH):
    ch1 = chemCH['Name'][i]
    temp = []
    for j in range(len(chemJP)):
        jp1 = chemJP['Name'][j]
        vectorizer = CountVectorizer().fit_transform([ch1, jp1])
        vectors = vectorizer.toarray()
        temp.append(ch1+' / '+jp1)
        temp.append(cosine_sim_vectors(vectors[0],vectors[1]))
    result.append(temp)
    i+=1

import operator
from collections import defaultdict
i = 0
result = []
while i < len(chemCH):
    ch1 = chemCH['Name'][i]
    temp = []
    dict = {}
    for j in range(len(chemJP)):
        jp1 = chemJP['Name'][j]
        vectorizer = CountVectorizer().fit_transform([ch1, jp1])
        vectors = vectorizer.toarray()
        dict[ch1+ '/ ' +jp1] = cosine_sim_vectors(vectors[0],vectors[1])
    result.append(dict)
    i+=1
for k in range(len(result)):
    maxCS = max(result[k], key=result[k].get)
    all_values = result[k].values()
    max_values = max(all_values)
    temp = []
    temp.append(maxCS)
    temp.append(max_values)
    print(temp)
    

In [15]:
#cosine similarity check of translated document against HP GSE terminology
i = 0
result = []
while i < len(chemCH):
    ch1 = chemCH['Name'][i]
    temp = []
    dict = {}
    for table in table_list:
        table.iloc[:,1] = table.iloc[:,1].str.replace('\n', '')
        table = table.reset_index(inplace = False)
        for j in range(len(table)):
            try:
                jp1 = table.iloc[j,1]
                vectorizer = CountVectorizer().fit_transform([ch1, jp1])
                vectors = vectorizer.toarray()
                dict[ch1+ '/' +jp1] = cosine_sim_vectors(vectors[0],vectors[1])
            except AttributeError:
                pass
    result.append(dict)
    i+=1

In [16]:
#return maximum cosine similarity which is assumed to be the best match
temp = []
for k in range(len(result)):
    maxCS = max(result[k], key=result[k].get)
    all_values = result[k].values()
    max_values = max(all_values)
    temp.append(maxCS)
    temp.append(max_values)
print(temp)
    #output format:external company naming/HP naming, cosine similarity

['Cadmium and cadmium compounds /Cadmium and its \ncompounds', 0.8164965809277261, 'Lead and lead compounds /Lead and its \ncompounds', 0.8164965809277261, 'Lead and lead compounds (applicable to covered wire) /Lead and its \ncompounds', 0.6324555320336759, 'Mercury and mercury compounds /Mercury and its \ncompounds', 0.8164965809277261, 'Hexavalent chromium compound /Hexavalent \nchromium and its \ncompounds', 0.5163977794943223, 'Polychlorinated biphenyls (PCB) /Polychlorinated \nbiphenyls (PCBs)', 0.6666666666666669, 'Polychlorinated Naphthalene (PCN) /Polychlorinated \nnaphthalenes', 0.408248290463863, 'Polychlorinated Terphenyl (PCT) /Polychlorinated \nnaphthalenes', 0.408248290463863, 'Short-chain chlorinated alkanes (SCCP) /Short chain \nchlorinated \nparaffins (SCCPs)', 0.6, 'Other organochlorine compounds /Other dibutyltin compounds', 0.6666666666666669, 'Polybrominated biphenyls (PBB) /Flame Retardant, \npolybrominated \nbiphenyls (PBBs)', 0.5163977794943223, 'Polybrominated 

In [18]:
#take the matched term to locate in HP GSE
returnlist = []
for i in range(len(temp)):
    if i % 2 ==0:
        temp[i] = temp[i].split('/')[1]
        #temp[i] = temp[i].replace('\n', '')
        returnlist.append(temp[i])
    #string = string.split('/')[1]

chemCH[chemCH['Name']=='Cadmium and cadmium compounds ']

for names in returnlist:
    chemCH[chemCH['Name']==names]

In [42]:
for names in returnlist:    
    for table in table_list:
        print(table[table.iloc[:,0] == names])     

Empty DataFrame
Columns: [, Document Identifier, , HX-00011-00]
Index: []
Empty DataFrame
Columns: [Table 1. Examples of Relevant Requirements for Product Types, None, None, None, None, None, None]
Index: []
Empty DataFrame
Columns: [•  Revision, 
Date, 
Change 
Number, •  Brief Description of change]
Index: []
Empty DataFrame
Columns: [T, 1-Jun-2015, •  Added new HP Standard 011-06 General Specification for the Environment – 
Manufacturing Process Substances Requirements 
•  Added Table 1 from HP Standard 011-01 
•  Added TBBPA to Table 1 
•  Updated Table 1 for 1,2,5,69,10-Hexabromocyclododecane (HBCDD or 
HBCD) in HP 011-01 document Table 1 and HP 011-01A document Table 1 
•  Updated Table 1 for PAH (also moved the PAH table 3 to 01 document) in 
HP 011-01 document Table 1Updated Table 1 for removal of “Bromine in 
printed circuit boards”]
Index: []
Empty DataFrame
Columns: [, Document Identifier, , HX-00011-01]
Index: []
  Table 1. Pan HP Mandatory Restrictions for All Products    

Index: []
                         Name CAS* Numbers
4  Other dibutyltin compounds            –
Empty DataFrame
Columns: [Name20, CAS* Numbers]
Index: []
Empty DataFrame
Columns: [Name, CAS* Numbers]
Index: []
Empty DataFrame
Columns: [Table 12. Class I substances, None, None, None, None, None, None]
Index: []
Empty DataFrame
Columns: [Table 13. Class II compounds, None, None, None, None, None, None]
Index: []
Empty DataFrame
Columns: [Table 14. Class III compounds, None, None, None, None, None, None]
Index: []
Empty DataFrame
Columns: [Revision, Date, 
Change Number, Brief Description of change]
Index: []
Empty DataFrame
Columns: [, Document Identifier, , HX-00011-02]
Index: []
Empty DataFrame
Columns: [Table 1. Recycled Content Requirements, None, None]
Index: []
Empty DataFrame
Columns: [Table 2. Korea Packaging Recyclability Evaluation Criteria, None, None, None]
Index: []
Empty DataFrame
Columns: [Table 3. Packaging Material Identifications Codes and Abbreviations, None, None]
Ind

4  None  
Empty DataFrame
Columns: [Revision, 
Date, 
Change Number, Brief Description of change]
Index: []
Empty DataFrame
Columns: [V, 3-Jul-2017, Added 3.1.6 Cobalt Dichloride 
Revised 3.2 Recyclable Materials 
Revised 3.4 Recycled and Certified Fiber Content in Paper-based Packaging 
Revised 3.5 Wood, Paper, and other Plant-based Packaging Requirements 
Added 3.5.1 Australian Illegal Logging Prohibition Act Import Declaration 
Added 3.7 Country-Specific Packaging Requirements  
Added 3.7.1 India 
Revised 4.1 General Material Coding]
Index: []
        Document Identifier       HX-00011-06
0         Revision and Date    F, 29-Jul-2020
1    Last Revalidation Date       29-Jul-2020
4                    Status          Approved
Empty DataFrame
Columns: [Table 1: Pan-HP Mandatory Restrictions for All Manufacturing Process Substances, None, None, None, None, None]
Index: []
  Table 2. VOC Content in Water-based Coatings    NaN                   NaN  \
2                                    